# Analysis

In [1]:
import os
import pandas as pd
from importlib import reload

# Get the current working directory
os.chdir("c:/Users/juanm_8qa8lav/Documents/Proyectos_Personales/FinancialApp")
# Verify the current working directory
print(f"Now in directory: {os.getcwd()}")

# Import your module
from app.modules.database.credit_manager import credits_balance

balance = credits_balance()
balance['D_Due'] = balance['D_Due'].dt.to_period('D')

print(f'Capital: $ {balance['Capital'].sum():,.2f}')
print(f'Interés: $ {balance['Interest'].sum():,.2f}')
print(f'IVA: $ {balance['IVA'].sum():,.2f}')
print(f'Total: $ {balance['Total'].sum():,.2f}')

Now in directory: c:\Users\juanm_8qa8lav\Documents\Proyectos_Personales\FinancialApp
Capital: $ 43,723,832.79
Interés: $ 39,623,640.79
IVA: $ 8,320,964.45
Total: $ 91,668,438.51


In [2]:
import tkinter as tk
from tkinter import filedialog

def seleccionar_archivo():
    """
    Opens a Windows file dialog in the foreground (always on top).
    Returns the full path to the selected file or an empty string if canceled.
    """
    # Create the root window
    root = tk.Tk()
    
    # Make the root window always on top
    root.attributes("-topmost", True)
    # Hide the root window so it's not visible
    root.withdraw()
    # Raise it to the front of the window stack
    root.lift()

    # Open the "Open File" dialog with the root as the parent
    ruta_archivo = filedialog.askopenfilename(
        title="Seleccionar un archivo",
        filetypes=[("All Files", "*.*")],
        parent=root
    )

    # Destroy the root window to free resources
    root.destroy()

    if ruta_archivo:
        print(f"Selected file: {ruta_archivo}")
    else:
        print("No file was selected.")
        
    # Return the selected path (or empty string if canceled)
    return ruta_archivo

path = seleccionar_archivo()

Selected file: C:/Users/juanm_8qa8lav/Documents/Proyectos_Personales/FinancialApp/inputs/Legajos/9080 - AMUF - LOPEZ CECILIA DE LAS MERCEDES.pdf


In [3]:
from app.modules.database.pdf_ocr_mistral import ask_question_about_pdf

respuesta = ask_question_about_pdf("docs/cedula_completa.pdf", "¿Cuál es la fecha de vencimiento?")
print(respuesta)


PDFPageCountError: Unable to get page count.
I/O Error: Couldn't open file 'docs/cedula_completa.pdf': No such file or directory.
